# Lecture 12 Supplementary Notebook

## DSC 40A, Fall 2024

The following cell sets up the necessary imports – don't worry too much about it.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats("svg")

pd.options.plotting.backend = "plotly"

# DSC 80 preferred styles
pio.templates["dsc80"] = go.layout.Template(
    layout=dict(
        margin=dict(l=30, r=30, t=30, b=30),
        autosize=True,
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
        title=dict(x=0.5, xanchor="center"),
    )
)
pio.templates.default = "simple_white+dsc80"

from IPython.display import HTML

Let's load in the sales dataset as a `pandas` DataFrame.

In [2]:
data = pd.read_csv('data/sales.csv')
data.head()

,net_sales,sq_ft,inventory,advertising,district_size,competing_stores
0,231.0,3.0,294,8.2,8.2,11
1,156.0,2.2,232,6.9,4.1,12
2,10.0,0.5,149,3.0,4.3,15
3,519.0,5.5,600,12.0,16.1,1
4,437.0,4.4,567,10.6,14.1,5


## Multiple Linear Regression

Using our linear algebraic formulation, the optimal intercept and slope are given by the vector $\vec{w}^*$, where:

$$\vec{w}^* = ({X^TX})^{-1} X^T\vec{y}$$

Here:
- $X$ is a $n \times 2$ matrix, called the **design matrix**, defined as:

$${ X} = \begin{bmatrix} { 1} & { x_1} \\ { 1} & { x_2} \\ \vdots & \vdots \\ { 1} & { x_n} \end{bmatrix}$$

- $\vec{y}$ is a $n$-dimensional vector, called the **observation vector**, defined as:

$$\vec{y} = \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}$$

### Using just one feature

Before we perform multiple linear regression, let's first just perform simple linear regression. We'll try and use square footage to predict net sales; our hypothesis function will be:

$$
\text{predicted net sales} = w_0 + w_1 \cdot \text{square feet}
$$

In [3]:
# pio.renderers.default = 'plotly_mimetype+notebook' # If the plot doesn't load for you, run this first.

In [4]:
fig = px.scatter(data, x='sq_ft', y='net_sales')
fig.update_layout(xaxis_title='Square Feet', yaxis_title='Net Sales', title='Net Sales vs. Square Footage')

It seems like $w_1^*$, the optimal slope, should be positive. To find $w_0^*$ and $w_1^*$, we'll solve the normal equations.

In [5]:
def solve_normal_equations(X, y):
    '''Returns the optimal parameter vector, w*, given a design matrix X and observation vector y.'''
    return np.linalg.solve(X.T @ X, X.T @ y)

In [6]:
data['1'] = 1

X_one_feature_model = data[['1', 'sq_ft']]
X_one_feature_model.to_numpy()

array([[1.        , 3.        ],
       [1.        , 2.20000005],
       [1.        , 0.5       ],
       [1.        , 5.5       ],
       [1.        , 4.4000001 ],
       [1.        , 4.80000019],
       [1.        , 3.0999999 ],
       [1.        , 2.5       ],
       [1.        , 1.20000005],
       [1.        , 0.60000002],
       [1.        , 5.4000001 ],
       [1.        , 4.19999981],
       [1.        , 4.69999981],
       [1.        , 0.60000002],
       [1.        , 1.20000005],
       [1.        , 1.60000002],
       [1.        , 4.30000019],
       [1.        , 2.5999999 ],
       [1.        , 3.79999995],
       [1.        , 5.30000019],
       [1.        , 5.5999999 ],
       [1.        , 0.80000001],
       [1.        , 1.10000002],
       [1.        , 3.5999999 ],
       [1.        , 3.5       ],
       [1.        , 5.0999999 ],
       [1.        , 8.6       ]])

In [7]:
y = data['net_sales']

In [8]:
w_one_feature_model = solve_normal_equations(X_one_feature_model, y)
w_one_feature_model

array([ 2.57700629, 85.38887328])

This is telling us that the best-fitting line to this dataset is

$$\text{predicted net sales} = 2.577 + 85.389 \cdot \text{square feet}$$

To get predictions for all observations in my dataset:

In [9]:
X_one_feature_model @ w_one_feature_model

,0
0,258.743626
1,190.432532
2,45.271443
3,472.215809
4,378.288057
5,412.443614
6,267.282505
7,216.049189
8,105.043658
9,53.810332


Let's draw a plot of our hypothesis function.

In [10]:
px.scatter(data, x='sq_ft', y='net_sales', title='Net Sales vs. Square Feet')

x_range = np.linspace(0, 10)

fig = go.Figure()
fig.add_trace(go.Scatter(x=data['sq_ft'], y=y, mode='markers', name='actual'))
fig.add_trace(go.Scatter(x=x_range,
                         y=w_one_feature_model[0] + w_one_feature_model[1] * x_range,
                         name='Linear hypothesis Function',
                         line=dict(color='red')))

fig.update_layout(xaxis_title='Square Feet', yaxis_title='Net Sales', title='Net Sales vs. Square Footage')

It's also worth calculating the mean squared error of this hypothesis function, so that we can compare it to our later hypothesis functions.

In [11]:
def mean_squared_error(X, y, w):
    return np.mean(np.sum((y - X @ w)**2))

mean_squared_error(X_one_feature_model, y, w_one_feature_model)

192390.89617894337

### Using two features

Let's now try to predict net sales from two variables: the square footage (size) of the store, and the number of competing stores in the area. Our model will be:

$$
\text{predicted net sales} = w_0 + w_1 \cdot \text{square feet} + w_2 \cdot \text{competitors}
$$

Suppose $w_0^*$, $w_1^*$, and $w_2^*$ are our hypothesis function's optimal parameters. Do you expect $w_1^*$ to be positive or negative? What about $w_2^*$?

In [12]:
fig = px.scatter(data, x='sq_ft', y='net_sales')
fig.update_layout(xaxis_title='Square Feet', yaxis_title='Net Sales', title='Net Sales vs. Square Footage')

In [15]:
fig = px.scatter(data, x='competing_stores', y='net_sales')
fig.update_layout(xaxis_title='Competing stores', yaxis_title='Net Sales', title='Net Sales vs. Number of Competing Stores')

Looking at separate scatter plots only tells part of the story. Let's look at a 3D scatter plot, with one axis for square footage, one axis for competing stores, and one axis for net sales.

In [16]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=data['sq_ft'],
                           y=data['competing_stores'],
                           z=data['net_sales'], mode='markers'))

fig.update_layout(scene=dict(
    xaxis_title='Square Footage',
    yaxis_title='Competing Stores',
    zaxis_title='Net Sales'),
    title='Net Sales vs. Square Footage and Number of Competing Stores')

Our goal is to find the best fitting **plane** to this set of points.

### Question 🤔

At the start of this notebook, we fit a hypothesis function with a single feature, square feet, and got that the weight of that feature was $w_1^* = 85.389$.

We are about to fit a hypothesis function with two features, square feet and competing stores.

**Question:** Is the weight of the square feet feature, $w_1^*$, for this **new** hypothesis function guaranteed to be equal to 85.389?

A. Yes

B. No

Our design matrix is:
    
$$
\begin{pmatrix}
 1 & s_1 & c_1\\
 1 & s_2 & c_2\\
 \vdots & \vdots & \vdots\\
 1 & s_n & c_n
\end{pmatrix}
$$

where $s_i$ is the size of the $i$th store, and $c_n$ is the number of competitors. In code:

In [17]:
X_two_feature_model = data[['1', 'sq_ft', 'competing_stores']].to_numpy()
X_two_feature_model

array([[ 1.        ,  3.        , 11.        ],
       [ 1.        ,  2.20000005, 12.        ],
       [ 1.        ,  0.5       , 15.        ],
       [ 1.        ,  5.5       ,  1.        ],
       [ 1.        ,  4.4000001 ,  5.        ],
       [ 1.        ,  4.80000019,  4.        ],
       [ 1.        ,  3.0999999 , 10.        ],
       [ 1.        ,  2.5       , 12.        ],
       [ 1.        ,  1.20000005, 15.        ],
       [ 1.        ,  0.60000002,  8.        ],
       [ 1.        ,  5.4000001 ,  1.        ],
       [ 1.        ,  4.19999981,  7.        ],
       [ 1.        ,  4.69999981,  3.        ],
       [ 1.        ,  0.60000002, 14.        ],
       [ 1.        ,  1.20000005, 11.        ],
       [ 1.        ,  1.60000002, 10.        ],
       [ 1.        ,  4.30000019,  4.        ],
       [ 1.        ,  2.5999999 , 13.        ],
       [ 1.        ,  3.79999995,  7.        ],
       [ 1.        ,  5.30000019,  1.        ],
       [ 1.        ,  5.5999999 ,  0.   

Using the function `solve_normal_equations` that we already built:

In [18]:
w_two_feature_model = solve_normal_equations(X_two_feature_model, y)
w_two_feature_model

array([303.49073761,  45.15092186, -21.5851804 ])

This is telling us that the best-fitting plane to this dataset is

$$\text{predicted net sales} = 303.491 + 45.151 \cdot \text{square feet} - 21.585 \cdot \text{competing stores}$$

**Note that the weight of $\text{square feet}$ in this hypothesis function is different than the weight of $\text{square feet}$ in the hypothesis function that only had one feature!**

In [19]:
XX, YY = np.mgrid[-1:10:2, 0:16:2]
Z = w_two_feature_model[0] + w_two_feature_model[1] * XX + w_two_feature_model[2] * YY
plane = go.Surface(x=XX, y=YY, z=Z, colorscale='Reds')

fig = go.Figure(data=[plane])
fig.add_trace(go.Scatter3d(x=data['sq_ft'],
                           y=data['competing_stores'],
                           z=data['net_sales'], mode='markers', marker={'color': '#656DF1'}))

fig.update_layout(scene=dict(
    xaxis_title='Square Footage',
    yaxis_title='Competing Stores',
    zaxis_title='Net Sales'),
    title='Net Sales vs. Square Footage and Number of Competing Stores')

As before, let's calculate the MSE:

In [20]:
mean_squared_error(X_two_feature_model, y, w_two_feature_model)

72287.04554905626

Note that this is significantly lower than the MSE of the model with just one feature:

In [21]:
mean_squared_error(X_one_feature_model, y, w_one_feature_model)

192390.89617894337

### All features

Let's fit a hypothesis function using all of the features.

In [22]:
column_order = ['1', 'sq_ft', 'competing_stores', 'inventory', 'advertising', 'district_size']
X_all_features = data[column_order].to_numpy()
X_all_features

array([[1.00000000e+00, 3.00000000e+00, 1.10000000e+01, 2.94000000e+02,
        8.19999981e+00, 8.19999981e+00],
       [1.00000000e+00, 2.20000005e+00, 1.20000000e+01, 2.32000000e+02,
        6.90000010e+00, 4.09999990e+00],
       [1.00000000e+00, 5.00000000e-01, 1.50000000e+01, 1.49000000e+02,
        3.00000000e+00, 4.30000019e+00],
       [1.00000000e+00, 5.50000000e+00, 1.00000000e+00, 6.00000000e+02,
        1.20000000e+01, 1.61000004e+01],
       [1.00000000e+00, 4.40000010e+00, 5.00000000e+00, 5.67000000e+02,
        1.06000004e+01, 1.41000004e+01],
       [1.00000000e+00, 4.80000019e+00, 4.00000000e+00, 5.71000000e+02,
        1.18000002e+01, 1.26999998e+01],
       [1.00000000e+00, 3.09999990e+00, 1.00000000e+01, 5.12000000e+02,
        8.10000000e+00, 1.01000004e+01],
       [1.00000000e+00, 2.50000000e+00, 1.20000000e+01, 3.47000000e+02,
        7.69999981e+00, 8.40000000e+00],
       [1.00000000e+00, 1.20000005e+00, 1.50000000e+01, 2.12000000e+02,
        3.29999995e+00, 

In [23]:
w_all_features = solve_normal_equations(X_all_features, y)
w_all_features

array([-18.85941416,  16.20157356,  -5.31097141,   0.17463515,
        11.52626903,  13.5803129 ])

In [24]:
for i, feature in enumerate(column_order):
    if feature == '1':
        print(f'intercept:\t{w_all_features[0]:0.3f}')
    else:
        print(f'{feature}:\t{w_all_features[i]:0.3f}')

intercept:	-18.859
sq_ft:	16.202
competing_stores:	-5.311
inventory:	0.175
advertising:	11.526
district_size:	13.580


The MSE of this model is even lower!

In [25]:
mean_squared_error(X_all_features, y, w_all_features)

6541.410343631843

Note that I can't visualize this hypothesis function, since I would need to be able to visualize in 6D, but I can still find this hypothesis function's predictions:

In [26]:
X_all_features @ w_all_features

array([228.54132342, 128.77828635,  28.57159462, 526.67763281,
       438.55165964, 445.8608775 , 298.19289184, 221.33815906,
        24.49589951,  86.4927322 , 568.5255428 , 423.92508128,
       468.73640753,   7.73991561,  70.48898947,  70.01098006,
       369.96817843, 156.99564616, 393.2790273 , 506.07015112,
       538.3281338 ,  88.84240462,  17.48878601, 352.21694784,
       347.13290225, 518.32948881, 411.92035997])

## Interpreting parameters

### Which feature is most "important"?

We should standardize in order to account for the difference in units and scale between the features.

**Question:** What would happen if I try to standardize the column of all 1s? 🧐

In [27]:
features = data[column_order].iloc[:, 1:].to_numpy()

In [28]:
standardized_features = (features - features.mean(axis=0)) / features.std(axis=0)

In [29]:
X_standardized = np.column_stack([
    np.ones(data.shape[0]),
    standardized_features
])

In [30]:
w_standardized = solve_normal_equations(X_standardized, y)
w_standardized

array([286.57407407,  31.97302867, -25.51529781,  32.76054166,
        42.69274551,  68.49841225])

In [31]:
for i, feature in enumerate(column_order):
    if feature == '1':
        print(f'intercept:\t{w_standardized[0]:0.3f}')
    else:
        print(f'{feature}:\t{w_standardized[i]:0.3f}')

intercept:	286.574
sq_ft:	31.973
competing_stores:	-25.515
inventory:	32.761
advertising:	42.693
district_size:	68.498


The district size appears to have the largest effect on the net sales.

In [32]:
mean_squared_error(X_standardized, y, w_standardized)

6541.410343631842

Note that standardizing has no impact on the actual predictions made by our hypothesis function, and hence the MSE – it just makes the weights more interpretable.